In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

import re

In [ ]:
def convert_google_sheet_url(url):
    # regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'
    # replace function to construct the new URL for CSV export
    # if gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'
    # replace using regex
    new_url = re.sub(pattern, replacement, url)
    return new_url

game_data_2024_url = 'https://docs.google.com/spreadsheets/d/1F8GBPtLhugdO0pqrJe3O9fMPOYv3AqVRq8ouUZWZxIY/edit#gid=0'
new_game_data_2024_url = convert_google_sheet_url(game_data_2024_url)
game_data_2024 = pd.read_csv(new_game_data_2024_url)

game_data_2023_url = 'https://docs.google.com/spreadsheets/d/1F8GBPtLhugdO0pqrJe3O9fMPOYv3AqVRq8ouUZWZxIY/edit#gid=256452955'
new_game_data_2023_url = convert_google_sheet_url(game_data_2023_url)
game_data_2023 = pd.read_csv(new_game_data_2023_url)

In [ ]:
game_data = pd.concat([game_data_2024, game_data_2023], ignore_index=True)

In [ ]:
game_data['count'] = game_data['balls'].astype(str).str.cat(game_data['strikes'].astype(str), sep ='-')

In [ ]:
game_data['count_advantage'] = 'neutral'
game_data.loc[(game_data['count'] == '1.0-0.0') | (game_data['count'] == '2.0-0.0') | (game_data['count'] == '2.0-1.0') | (game_data['count'] == '3.0-0.0') | (game_data['count'] == '3.0-1.0'), 'count_advantage'] = 'batter'
game_data.loc[(game_data['count'] == '0.0-1.0') | (game_data['count'] == '0.0-2.0') | (game_data['count'] == '1.0-2.0'), 'count_advantage'] = 'pitcher'

In [ ]:
game_data['CH'] = game_data['pitch_type'] == 'CH'
game_data['CH'] = game_data['CH'].replace({True: 1, False: 0})

game_data['CU'] = game_data['pitch_type'] == 'CU'
game_data['CU'] = game_data['CU'].replace({True: 1, False: 0})

game_data['SL'] = game_data['pitch_type'] == 'SL'
game_data['SL'] = game_data['SL'].replace({True: 1, False: 0})

game_data['FF'] = game_data['pitch_type'] == 'FF'
game_data['FF'] = game_data['FF'].replace({True: 1, False: 0})

game_data['SI'] = game_data['pitch_type'] == 'SI'
game_data['SI'] = game_data['SI'].replace({True: 1, False: 0})

game_data['CR'] = game_data['pitch_type'] == 'CR'
game_data['CR'] = game_data['CR'].replace({True: 1, False: 0})

In [ ]:
game_data['FsB'] = game_data['pitch_type'].isin(['FF', 'FT', 'SI', 'FC'])
game_data['FsB'] = game_data['FsB'].replace({True: 1, False: 0})

game_data['OfS'] = game_data['pitch_type'].isin(['CH', 'CR', 'FS', 'FO'])
game_data['OfS'] = game_data['OfS'].replace({True: 1, False: 0})

game_data['BrB'] = game_data['pitch_type'].isin(['CU', 'SL', 'KC', 'ST', 'SV'])
game_data['BrB'] = game_data['BrB'].replace({True: 1, False: 0})

game_data['OtR'] = game_data['pitch_type'].isin(['PO', 'KN', 'SC'])
game_data['OtR'] = game_data['OtR'].replace({True: 1, False: 0})

In [ ]:
game_data['spray_angle'] = np.nan
# use np.arctan2 then convert to degrees
game_data.loc[(game_data['hc_x'].isna() == False) & (game_data['hc_y'].isna() == False), 'spray_angle'] = np.arctan2(game_data['hc_x'], game_data['hc_y']) * 180 / np.pi

In [ ]:
game_data['hit_field_direction'] = None
game_data.loc[(game_data['spray_angle'] >= 105) | (game_data['spray_angle'] <= -90), 'hit_field_direction'] = 'left'
game_data.loc[(game_data['spray_angle'] < 105) & (game_data['spray_angle'] > 75), 'hit_field_direction'] = 'center'
game_data.loc[(game_data['spray_angle'] <= 75) | ((game_data['spray_angle'] < 0) & (game_data['spray_angle'] > -90)), 'hit_field_direction'] = 'right'

In [ ]:
game_data['hit_direction'] = None
game_data.loc[game_data['hit_field_direction'] == 'center'] = 'cent'
game_data.loc[((game_data['stand'] == 'R') & (game_data['hit_field_direction'] == 'left')) | ((game_data['stand'] == 'L') & (game_data['hit_field_direction'] == 'right')), 'hit_direction'] = 'pull'
game_data.loc[((game_data['stand'] == 'L') & (game_data['hit_field_direction'] == 'left')) | ((game_data['stand'] == 'R') & (game_data['hit_field_direction'] == 'right')), 'hit_direction'] = 'oppo'

In [ ]:
game_data['BIP'] = game_data['type'] == 'X'
game_data['BIP'] = game_data['BIP'].replace({True: 1, False: 0})

game_data['pull'] = game_data['hit_direction'] == 'pull'
game_data['pull'] = game_data['pull'].replace({True: 1, False: 0})

game_data['cent'] = game_data['hit_direction'] == 'cent'
game_data['cent'] = game_data['cent'].replace({True: 1, False: 0})

game_data['oppo'] = game_data['hit_direction'] == 'oppo'
game_data['oppo'] = game_data['oppo'].replace({True: 1, False: 0})